# Heating Scheduler Example

Once you retrieved your authentication information from the MIA app, you can setup the client and begin with an initial device/room discovery:

In [ ]:
import logging

from environs import Env

import iolite

from iolite.client import IOLiteClient
from iolite.oauth_handler import OAuthHandler, OAuthStorage, OAuthWrapper

env = Env()
env.read_env()

USERNAME = env("HTTP_USERNAME")
PASSWORD = env("HTTP_PASSWORD")
CODE = env("CODE")
NAME = env("NAME")
LOG_LEVEL = env.log_level("LOG_LEVEL", logging.INFO)

logging.basicConfig(level=logging.getLevelName(LOG_LEVEL))
logger = logging.getLogger(__name__)


# Get SID
oauth_storage = OAuthStorage("../")
oauth_handler = OAuthHandler(USERNAME, PASSWORD)
oauth_wrapper = OAuthWrapper(oauth_handler, oauth_storage)
sid = oauth_wrapper.get_sid(CODE, NAME)

client = IOLiteClient(sid, USERNAME, PASSWORD)
client.discover()

Once the discovery is complete (when you see the "QuerySuccess" message), you can manipulate the heating intervals using the HeatingScheduler.

Note: If you renamed the rooms in your Mia app, the room name in the client name may not be the same as shown in the app. Use client.discovered.get_rooms() instead, to get a list of all rooms.

In [ ]:
from iolite.heating_scheduler import HeatingScheduler, Day
room = client.discovered.find_room_by_name("Raum1")
heating_scheduler = HeatingScheduler(sid, USERNAME, PASSWORD, room.identifier)

So far, you can set the comfort temperature:

In [ ]:
response = heating_scheduler.set_comfort_temperature(20.5)
response

Add new heating intervals:

In [ ]:
# add a new interval on Sunday lasting from 2am to 4am - be careful not to overlap with existing intervals,
# otherwise existing intervals will be merged into the newly created interval.
# This also applies to "nearby" intervals (e.g. an existing interval beginning at 7 will be merged with a new interval ending at 7)
response = heating_scheduler.add_interval(Day.SUNDAY, 2, 0, 120)
created = response.json()["id"]
response

And delete existing intervals (here we delete the interval we just created):

In [ ]:
response = heating_scheduler.delete_interval(created)
response